In [1]:
import os
import requests
import jwt
import time
import sqlite3
import json
import re
from datetime import datetime , timedelta

def save_to_file(data, file_path):
    with open(file_path, 'w') as file:
        # file.write(data)
        json.dump(data, file)

def load_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def generate_jwt_token(client_id, client_secret):
    # Replace with the appropriate token endpoint URL for your OAuth server
    token_endpoint = "https://web.arbeitsagentur.de/ausbildungssuche/berufsausbildung-suche?sty=0&atyp=102&kat=0"

    now = int(time.time())
    payload = {
        "iss": client_id,
        "sub": client_id,
        "aud": token_endpoint,
        "iat": now,
        "exp": now + 3600,  # Token expires in 1 hour
    }

    # Sign the payload with your client secret to generate the JWT
    jwt_token = jwt.encode(payload, client_secret, algorithm='HS256')

    return jwt_token

def get_access_token(client_id, client_secret):
    # Generate the JWT token
    jwt_token = generate_jwt_token(client_id, client_secret)

    # Request the access token using the JWT token
    token_endpoint = "https://rest.arbeitsagentur.de/oauth/gettoken_cc"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    response = requests.post(token_endpoint, headers=headers, data=data)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to get access token. Status code: {response.status_code}")

def get_api_data(api_url, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}",
    }

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data:  # Check if data is not empty
            return data
        else:
            print("No more data")
    else:
        raise Exception(f"Failed to fetch data from API. Status code: {response.status_code}")
    
def create_table(db_connection):
    cursor = db_connection.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS schulisch_ausbildung
                            (
                                veranstaltungs_ID INT PRIMARY KEY,
                                trainingstitel TEXT NULL,
                                dauer TEXT,
                                unterrichtsform TEXT,
                                abschlussart TEXT,
                                abschlussbezeichnung TEXT,
                                finanzielle_unterstützung  TEXT,
                                zugangsinformationen TEXT,
                                zielgruppe TEXT,
                                kredit TEXT,
                                anbieter_ID INT,
                                bildungsanbieter TEXT,
                                telefonVorwahl INT,
                                telefonDurchwahl INT,
                                mobilVorwahl INT,
                                mobilDurchwahl  INT,
                                faxVorwahl INT,
                                faxDurchwahl INT,
                                homepage TEXT,
                                email TEXT,
                                latitude REAL,
                                longitude REAL,
                                stadt TEXT,
                                land TEXT,
                                schulart TEXT,
                                unterrichtszeiten TEXT,
                                kostenWert TEXT,
                                kostenWaehrung TEXT,
                                individuellerEinstieg TEXT,
                                foerderung TEXT,
                                link TEXT,
                                beginn DATE,
                                ende DATE,
                                teilnehmerMin INT,
                                teilnehmerMax INT,
                                aktualisierungsdatum DATE 
                                )
                ''')
    db_connection.commit()

    
def insert_data_into_db(data, db_connection):
    #print(len(data))
    cursor = db_connection.cursor()
    for entry in data:
        
        veranstaltungs_ID = entry["id"]
        trainingstitel = entry["angebot"]["titel"]
        dauer = entry["dauer"]["bezeichnung"]
        unterrichtsform = entry["unterrichtsform"]["bezeichnung"]
        abschlussart = re.sub(r'<[^>]+>', '', str(entry["angebot"]["abschlussart"])) 
        abschlussbezeichnung = re.sub(r'<[^>]+>', '', str(entry["angebot"]["abschlussbezeichnung"])) 
        finanzielle_unterstützung = re.sub(r'<[^>]+>', '', str(entry["angebot"]["foerderung"])) 
        zugangsinformationen = re.sub(r'<[^>]+>', '', str(entry["angebot"]["zugang"])) 
        zielgruppe = re.sub(r'<[^>]+>', '', str(entry["angebot"]["zielgruppe"]))
        kredit = re.sub(r'<[^>]+>', '', str(entry["angebot"]["anrechnung"]))
        anbieter_ID = entry["angebot"]["bildungsanbieter"]["id"]
        bildungsanbieter = entry["angebot"]["bildungsanbieter"]["name"]
        telefonVorwahl = entry["angebot"]["bildungsanbieter"]["telefonVorwahl"]
        telefonDurchwahl = entry["angebot"]["bildungsanbieter"]["telefonDurchwahl"]
        mobilVorwahl = entry["angebot"]["bildungsanbieter"]["mobilVorwahl"]
        mobilDurchwahl  = entry["angebot"]["bildungsanbieter"]["mobilDurchwahl"]
        faxVorwahl = entry["angebot"]["bildungsanbieter"]["faxVorwahl"]
        faxDurchwahl = entry["angebot"]["bildungsanbieter"]["faxDurchwahl"]
        homepage = entry["angebot"]["bildungsanbieter"]["homepage"]
        email = entry["angebot"]["bildungsanbieter"]["email"]
        latitude = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["koordinatenPlz"]["lat"]
        longitude = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["koordinatenPlz"]["lon"]
        stadt = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["name"]
        land = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["land"]["name"]
        schulart = entry["angebot"]["schulart"]["bezeichnung"]
        unterrichtszeiten = re.sub(r'<[^>]+>', '', str(entry["unterrichtszeiten"]))
        kostenWert = entry["kostenWert"]
        kostenWaehrung = entry["kostenWaehrung"]
        individuellerEinstieg = "ja" if entry["individuellerEinstieg"] == 1 else "nein"
        foerderung = "ja" if entry["foerderung"] == 1 else "nein"
        link = entry["link"]
        begin_timestamp = entry.get("beginn")
        if begin_timestamp is not None:
            beginn = (datetime.utcfromtimestamp(begin_timestamp / 1000)+ timedelta(days=1)).strftime('%Y-%m-%d')
        else:
            beginn = None

        end_timestamp = entry.get("ende")
        if end_timestamp is not None:
            ende = (datetime.utcfromtimestamp(end_timestamp / 1000)+ timedelta(days=1)).strftime('%Y-%m-%d')
        else:
            ende = None
        teilnehmerMin = entry["teilnehmerMin"]
        teilnehmerMax = entry["teilnehmerMax"]
        update_timestamp = entry["aktualisierungsdatum"]
        if update_timestamp is not None:
            aktualisierungsdatum = (datetime.utcfromtimestamp(update_timestamp / 1000)+ timedelta(days=1)).strftime('%Y-%m-%d')
        else:
            aktualisierungsdatum = None

        # Check if the event_id already exists in the table
        cursor.execute('''SELECT COUNT(*) FROM schulisch_ausbildung WHERE veranstaltungs_ID = ?''', (veranstaltungs_ID,))
        count = cursor.fetchone()[0]
        
        if count == 0: 
        # Insert data into the table
          cursor.execute('''INSERT INTO schulisch_ausbildung (
                                veranstaltungs_ID ,
                                trainingstitel ,
                                dauer ,
                                unterrichtsform ,
                                abschlussart ,
                                abschlussbezeichnung ,
                                finanzielle_unterstützung  ,
                                zugangsinformationen ,
                                zielgruppe ,
                                kredit ,
                                anbieter_ID ,
                                bildungsanbieter ,
                                telefonVorwahl ,
                                telefonDurchwahl ,
                                mobilVorwahl ,
                                mobilDurchwahl  ,
                                faxVorwahl ,
                                faxDurchwahl ,
                                homepage ,
                                email ,
                                latitude ,
                                longitude ,
                                stadt ,
                                land ,
                                schulart ,
                                unterrichtszeiten ,
                                kostenWert ,
                                kostenWaehrung ,
                                individuellerEinstieg ,
                                foerderung ,
                                link ,
                                beginn ,
                                ende ,
                                teilnehmerMin ,
                                teilnehmerMax ,
                                aktualisierungsdatum  
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                    veranstaltungs_ID ,
                    trainingstitel ,
                    dauer ,
                    unterrichtsform ,
                    abschlussart ,
                    abschlussbezeichnung ,
                    finanzielle_unterstützung  ,
                    zugangsinformationen ,
                    zielgruppe ,
                    kredit ,
                    anbieter_ID ,
                    bildungsanbieter ,
                    telefonVorwahl ,
                    telefonDurchwahl ,
                    mobilVorwahl ,
                    mobilDurchwahl  ,
                    faxVorwahl ,
                    faxDurchwahl ,
                    homepage ,
                    email ,
                    latitude ,
                    longitude ,
                    stadt ,
                    land ,
                    schulart ,
                    unterrichtszeiten ,
                    kostenWert ,
                    kostenWaehrung ,
                    individuellerEinstieg ,
                    foerderung ,
                    link ,
                    beginn ,
                    ende ,
                    teilnehmerMin ,
                    teilnehmerMax ,
                    aktualisierungsdatum  
            ))
    db_connection.commit()
    #print("insert data")
    
def main():
    api_url = "https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0" # Replace with the API endpoint URL
    data_file = "data_json.txt"
    client_id = "1c852184-1944-4a9e-a093-5cc078981294"  # Replace with your OAuth client ID
    client_secret = "777f9915-9f0d-4982-9c33-07b5810a3e79"  # Replace with your OAuth client secret

     # Get the access token using client credentials flow with JWTs
    access_token = get_access_token(client_id, client_secret)

    try:
        # Open a connection
        db_connection = sqlite3.connect('weiterbildung_analysis.db')
        
        # Create the table if not exists
        create_table(db_connection)

        # Pagination: Fetch all data from the API using multiple requests
        all_data = []
        initial_api_data = get_api_data(api_url, access_token)
        total_pages = initial_api_data["page"]["totalPages"]
        page = 0
        while page < total_pages:
            paginated_api_url = f"{api_url}&page={page}&size=20"  # Updated pagination URL
            print(paginated_api_url)
            api_data = get_api_data(paginated_api_url, access_token)
            
            if "_embedded" in api_data:
                insert_data_into_db(list(api_data["_embedded"]["termine"]), db_connection)
            else:
                print("No more data")
                break
            
            page += 1    

        # Insert data into the database
        insert_data_into_db(all_data, db_connection)

    except Exception as e:
        print(f"Error: {e}")
        db_connection.rollback()  # Rollback changes in case of an error
    
    finally:
        # Close the connection
        db_connection.close()
        print("Data fetched and saved to SQLite database successfully.")


if __name__ == "__main__":
    main()


https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=0&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=1&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=2&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=3&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=4&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=5&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=6&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=7&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&sty=0&page=8&size=20
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1